<a href="https://colab.research.google.com/github/hail-members/distributed-deep-learning/blob/main/Parallel%20computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import time

def matmul_original(A, B):
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
    return C

def matmul_reordered(A, B):
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(C.shape[0]):
        for k in range(A.shape[1]):
            for j in range(C.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
    return C

def compare_performance(A, B):
    # Original order (i-j-k)
    start_time = time.time()
    matmul_original(A, B)
    original_time = time.time() - start_time

    # Reordered (i-k-j)
    start_time = time.time()
    matmul_reordered(A, B)
    reordered_time = time.time() - start_time

    print(f"Original order time: {original_time:.6f} seconds")
    print(f"Reordered time: {reordered_time:.6f} seconds")
    print(f"Speedup: {original_time / reordered_time:.2f}x")

# Test the performance
if __name__ == "__main__":
    # Create random matrices
    A = np.random.rand(200, 200)
    B = np.random.rand(200, 200)

    print("Comparing performance for 200x200 matrices:")
    compare_performance(A, B)

    # Verify correctness
    C_original = matmul_original(A, B)
    C_reordered = matmul_reordered(A, B)
    C_numpy = np.dot(A, B)

    print("\nVerifying correctness:")
    print(f"Original vs NumPy: {np.allclose(C_original, C_numpy)}")
    print(f"Reordered vs NumPy: {np.allclose(C_reordered, C_numpy)}")

Comparing performance for 200x200 matrices:
Original order time: 7.022731 seconds
Reordered time: 5.456274 seconds
Speedup: 1.29x

Verifying correctness:
Original vs NumPy: True
Reordered vs NumPy: True


In [9]:
import numpy as np
import time

def matmul_original(A, B):
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
    return C

def matmul_tiled(A, B, tile_size=50):
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(0, C.shape[0], tile_size):
        for j in range(0, C.shape[1], tile_size):
            for k in range(0, A.shape[1], tile_size):
                # 각 타일에 대해 행렬 곱 수행
                for ii in range(i, min(i + tile_size, C.shape[0])):
                    for jj in range(j, min(j + tile_size, C.shape[1])):
                        for kk in range(k, min(k + tile_size, A.shape[1])):
                            C[ii, jj] += A[ii, kk] * B[kk, jj]
    return C

def compare_performance(A, B):
    # Original version
    start_time = time.time()
    matmul_original(A, B)
    original_time = time.time() - start_time

    # Tiled version
    start_time = time.time()
    matmul_tiled(A, B)
    tiled_time = time.time() - start_time

    print(f"Original version time: {original_time:.6f} seconds")
    print(f"Tiled version time: {tiled_time:.6f} seconds")
    print(f"Speedup: {original_time / tiled_time:.2f}x")

# Test the performance
if __name__ == "__main__":
    # Create random matrices
    A = np.random.rand(200, 200)
    B = np.random.rand(200, 200)

    print("Comparing performance for 500x500 matrices:")
    compare_performance(A, B)

    # Verify correctness
    C_original = matmul_original(A, B)
    C_tiled = matmul_tiled(A, B)
    C_numpy = np.dot(A, B)

    print("\nVerifying correctness:")
    print(f"Original vs NumPy: {np.allclose(C_original, C_numpy)}")
    print(f"Tiled vs NumPy: {np.allclose(C_tiled, C_numpy)}")

Comparing performance for 500x500 matrices:
Original version time: 7.321975 seconds
Tiled version time: 5.673241 seconds
Speedup: 1.29x

Verifying correctness:
Original vs NumPy: True
Tiled vs NumPy: True


In [10]:
import numpy as np
import time

def matmul_original(A, B):
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
    return C

def matmul_tiled(A, B, tile_size=100):
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(0, C.shape[0], tile_size):
        for j in range(0, C.shape[1], tile_size):
            for k in range(0, A.shape[1], tile_size):
                for ii in range(i, min(i + tile_size, C.shape[0])):
                    for jj in range(j, min(j + tile_size, C.shape[1])):
                        for kk in range(k, min(k + tile_size, A.shape[1])):
                            C[ii, jj] += A[ii, kk] * B[kk, jj]
    return C

def matmul_tiled_unrolled(A, B, tile_size=100, unroll_factor=10):
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(0, C.shape[0], tile_size):
        for j in range(0, C.shape[1], tile_size):
            for k in range(0, A.shape[1], tile_size):
                for ii in range(i, min(i + tile_size, C.shape[0])):
                    for jj in range(j, min(j + tile_size, C.shape[1])):
                        kk = k
                        while kk < min(k + tile_size, A.shape[1]) - unroll_factor + 1:
                            C[ii, jj] += A[ii, kk] * B[kk, jj]
                            C[ii, jj] += A[ii, kk+1] * B[kk+1, jj]
                            C[ii, jj] += A[ii, kk+2] * B[kk+2, jj]
                            C[ii, jj] += A[ii, kk+3] * B[kk+3, jj]
                            kk += unroll_factor
                        # Handle remaining elements
                        while kk < min(k + tile_size, A.shape[1]):
                            C[ii, jj] += A[ii, kk] * B[kk, jj]
                            kk += 1
    return C

def compare_performance(A, B):
    # Original version
    start_time = time.time()
    C_original = matmul_original(A, B)
    original_time = time.time() - start_time

    # Tiled version
    start_time = time.time()
    C_tiled = matmul_tiled(A, B)
    tiled_time = time.time() - start_time

    # Tiled and unrolled version
    start_time = time.time()
    C_tiled_unrolled = matmul_tiled_unrolled(A, B)
    tiled_unrolled_time = time.time() - start_time

    print(f"Original version time: {original_time:.6f} seconds")
    print(f"Tiled version time: {tiled_time:.6f} seconds")
    print(f"Tiled and unrolled version time: {tiled_unrolled_time:.6f} seconds")
    print(f"Speedup (Tiled vs Original): {original_time / tiled_time:.2f}x")
    print(f"Speedup (Tiled+Unrolled vs Original): {original_time / tiled_unrolled_time:.2f}x")

    # Verify correctness
    C_numpy = np.dot(A, B)
    print("\nVerifying correctness:")
    print(f"Original vs NumPy: {np.allclose(C_original, C_numpy)}")
    print(f"Tiled vs NumPy: {np.allclose(C_tiled, C_numpy)}")
    print(f"Tiled+Unrolled vs NumPy: {np.allclose(C_tiled_unrolled, C_numpy)}")

# Test the performance
if __name__ == "__main__":
    # Create random matrices
    A = np.random.rand(200, 200)
    B = np.random.rand(200, 200)

    print("Comparing performance for 200x200 matrices:")
    compare_performance(A, B)

Comparing performance for 200x200 matrices:
Original version time: 6.962770 seconds
Tiled version time: 5.540080 seconds
Tiled and unrolled version time: 3.924790 seconds
Speedup (Tiled vs Original): 1.26x
Speedup (Tiled+Unrolled vs Original): 1.77x

Verifying correctness:
Original vs NumPy: True
Tiled vs NumPy: True
Tiled+Unrolled vs NumPy: False


In [11]:
N = 1000000
A = np.random.rand(N)
B = np.random.rand(N)

# SISD
start_sisd = time.time()
C_sisd = np.zeros(N)
for i in range(N):
    C_sisd[i] = A[i] * B[i]
end_sisd = time.time()
sisd_time = end_sisd - start_sisd

# Vectorized data
start_vectorized = time.time()
C_vectorized = A * B
end_vectorized = time.time()
vectorized_time = end_vectorized - start_vectorized

# Speedup calculation
speedup = sisd_time / vectorized_time


print(f"Original (SISD) version time: {sisd_time:.6f} seconds")
print(f"Vectorized version time: {vectorized_time:.6f} seconds")
print(f"Speedup: {sisd_time / vectorized_time:.2f}x")

Original (SISD) version time: 0.513688 seconds
Vectorized version time: 0.007482 seconds
Speedup: 68.65x


In [13]:
from multiprocessing import Pool

# Matrix multiplication without multiprocessing
def matrix_multiply(A, B):
    N = len(A)
    C = [[0] * N for _ in range(N)]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                C[i][j] += A[i][k] * B[k][j]
    return C

# Matrix multiplication with multiprocessing
def parallel_multiply_row(args):
    i, A, B = args
    N = len(A)
    row = [0] * N
    for j in range(N):
        for k in range(N):
            row[j] += A[i][k] * B[k][j]
    return row

def matrix_multiply_parallel(A, B, num_processes=4):
    N = len(A)
    pool = Pool(processes=num_processes)
    result = pool.map(parallel_multiply_row, [(i, A, B) for i in range(N)])
    pool.close()
    pool.join()
    return result

# Test function to compare times
def compare_times(N=100):
    # Create two NxN matrices filled with random integers
    A = [[np.random.randint(0, 10) for _ in range(N)] for _ in range(N)]
    B = [[np.random.randint(0, 10) for _ in range(N)] for _ in range(N)]

    # Without multiprocessing
    start = time.time()
    C_single = matrix_multiply(A, B)
    end = time.time()
    single_time = end - start

    # With multiprocessing
    start = time.time()
    C_parallel = matrix_multiply_parallel(A, B)
    end = time.time()
    parallel_time = end - start

    # With numpy
    A_np = np.array(A)
    B_np = np.array(B)
    start = time.time()
    C_np = np.dot(A_np, B_np)
    end = time.time()
    numpy_time = end - start

    return single_time, parallel_time, numpy_time

single_time_reduced, parallel_time_reduced, numpy_time_reduced = compare_times(N=100)

# Calculate speedup for reduced matrix size
parallel_speedup_reduced = single_time_reduced / parallel_time_reduced
numpy_speedup_reduced = single_time_reduced / numpy_time_reduced

print(f"Original version time: {single_time_reduced:.6f} seconds")
print(f"Multithreading version time: {parallel_time_reduced:.6f} seconds")
print(f"Speedup: {single_time_reduced / parallel_time_reduced:.2f}x")

print(f"Numpy version time: {numpy_time_reduced:.6f} seconds")
print(f"Numpy Speedup: {single_time_reduced / numpy_time_reduced:.2f}x")


# matrix size to 500x500 for feasible computation time
single_time_reduced, parallel_time_reduced, numpy_time_reduced = compare_times(N=500)

print(f"Original version time: {single_time_reduced:.6f} seconds")
print(f"Multithreading version time: {parallel_time_reduced:.6f} seconds")
print(f"Speedup: {single_time_reduced / parallel_time_reduced:.2f}x")

print(f"Numpy version time: {numpy_time_reduced:.6f} seconds")
print(f"Numpy Speedup: {single_time_reduced / numpy_time_reduced:.2f}x")


Original version time: 0.219301 seconds
Multithreading version time: 0.293322 seconds
Speedup: 0.75x
Numpy version time: 0.001040 seconds
Numpy Speedup: 210.87x
Original version time: 33.256274 seconds
Multithreading version time: 28.833416 seconds
Speedup: 1.15x
Numpy version time: 0.171654 seconds
Numpy Speedup: 193.74x
